In [1]:
import torch
import tensorflow as tf
from transformers import BertTokenizer, BertModel, TFBertModel, BertForMaskedLM 

import warnings; warnings.filterwarnings('ignore')

To use data.metrics please install scikit-learn. See https://scikit-learn.org/stable/index.html


# 0. Choose SweBERT model

In [2]:
pretrained_model_name = 'af-ai-center/bert-base-swedish-uncased'
# pretrained_model_name = af-ai-center/bert-large-swedish-uncased

# 1. Check SweBERT Model Accessibility

### a. Tokenizer

In [3]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name, do_lower_case=False)

### b. Model PyTorch

In [4]:
model = BertModel.from_pretrained(pretrained_model_name)

### c. Model TensorFlow

In [5]:
model = TFBertModel.from_pretrained(pretrained_model_name)

# 2. Simple Model Application (Masked Token Prediction)

In [6]:
example = 'Jag är ett barn, och det här är mitt hem. Alltså är det ett barnhem!'
example

'Jag är ett barn, och det här är mitt hem. Alltså är det ett barnhem!'

### 1. Preprocess the example

#### a. lowercase 

In [7]:
example_uncased = example.lower()
example_uncased

'jag är ett barn, och det här är mitt hem. alltså är det ett barnhem!'

#### b. special tokens 

In [8]:
example_preprocessed = '[CLS] ' + example_uncased.replace('.', '. [SEP]') + ' [SEP]'
example_preprocessed

'[CLS] jag är ett barn, och det här är mitt hem. [SEP] alltså är det ett barnhem! [SEP]'

### 2. Tokenize the preprocessed example

In [9]:
tokenizer = BertTokenizer.from_pretrained(pretrained_model_name, do_lower_case=False)  # see

In [10]:
tokens = tokenizer.tokenize(example_preprocessed)

print(f'{len(tokens)} tokens:')
print(tokens)

22 tokens:
['[CLS]', 'jag', 'är', 'ett', 'barn', ',', 'och', 'det', 'här', 'är', 'mitt', 'hem', '.', '[SEP]', 'alltså', 'är', 'det', 'ett', 'barn', '##hem', '!', '[SEP]']


### 3. Mask one of the tokens

In [11]:
masked_index = 18  # 'barn'
tokens[masked_index] = '[MASK]'

print(tokens)

['[CLS]', 'jag', 'är', 'ett', 'barn', ',', 'och', 'det', 'här', 'är', 'mitt', 'hem', '.', '[SEP]', 'alltså', 'är', 'det', 'ett', '[MASK]', '##hem', '!', '[SEP]']


### 4. Prepare the tokens for use with SweBERT

In [12]:
indexed_tokens = tokenizer.convert_tokens_to_ids(tokens)

In [13]:
indexed_tokens_tensor = torch.tensor([indexed_tokens])
print(indexed_tokens_tensor)

tensor([[ 101, 1112, 1100, 1115, 1255, 1010, 1095, 1102, 1174, 1100, 1352, 1345,
         1012,  102, 1492, 1100, 1102, 1115,  103, 2760,  999,  102]])


### 5. Use SweBERT to predict back the masked token

In [14]:
# instantiate model
model = BertForMaskedLM.from_pretrained(pretrained_model_name)
_ = model.eval()

In [15]:
# predict all tokens
with torch.no_grad():
    outputs = model(indexed_tokens_tensor)

predictions = outputs[0]
print(predictions.shape)  # 1 example, 21 tokens, 30522 vocab size

torch.Size([1, 22, 30522])


In [16]:
# show predicted index for masked token
predicted_index = torch.argmax(predictions[0, masked_index])
print(predicted_index)

tensor(1255)


In [17]:
# show predicted masked token
predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
print(predicted_token)

barn


In [18]:
assert predicted_token == 'barn'

#### Appendix

In [19]:
# show top5 predicted index for masked token
predicted_index_top5 = torch.argsort(predictions[0, masked_index], descending=True)[:5]
predicted_index_top5

tensor([ 1255, 14829,  8032,  1251,  1264])

In [20]:
# show top5 predicted masked token
for predicted_index in predicted_index_top5:
    predicted_token = tokenizer.convert_ids_to_tokens([predicted_index])[0]
    print(predicted_token)

barn
barndoms
foster
dock
dag


# Conclusions